In [401]:
from dataclasses import dataclass, field
import numpy as np
from tqdm import tqdm
from CifFile import ReadCif
from src.utils_py.gro.Structure import Structure
from src.utils_py.gro.Atom import Atom
from src.utils_py.io.gro import write_gro, read_gro
from src.utils_py.assembler.pbc import apply_pbc_to_points
from collections import defaultdict
import os

In [78]:
cf = ReadCif('CaCO3.cif')

In [79]:
for key in cf.keys():
    print(key)

caco3


---

# Полный цикл создания подложки

In [565]:
folder = 'cal_104_fixed'
l = np.array([100.13602, 15.02040, 101.94077]) * 0.1 # nm
angle = np.array([90, 90, 120])

In [403]:
os.system(f'gmx editconf -f {folder}/CaCO3_substrate.pdb -box {l[0]} {l[1]} {l[2]} -angles 90 90 120 -o {folder}/CaCO3_substrate_init.gro')

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 17900 atoms
No velocities found
    system size : 11.523  2.103 10.477 (nm)
    center      :  4.631  0.650  5.097 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  0.000 -0.000 -0.000 (nm)
new center      :  4.631  0.650  5.097 (nm)
new box vectors : 10.014  1.502 10.194 (nm)
new box angles  :  90.00  90.00 120.00 (degrees)
new box volume  : 132.79               (nm^3)


                 :-) GROMACS - gmx editconf, 2023-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2023/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2023
Working dir:  /Users/alexey/Dev/MD_conf
Command line:
  gmx editconf -f xz_thin/CaCO3_substrate.pdb -box 10.013602 1.50204 10.194077 -angles 90 90 120 -o xz_thin/CaCO3_substrate_init.gro


GROMACS reminds you: "Religion is a culture of faith; science is a culture of doubt." (Richard Feynman)



0

In [488]:
os.system(f'gmx editconf -f {folder}/CaCO3_substrate.pdb -o {folder}/CaCO3_substrate_init.gro')

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 9648 atoms
No velocities found

See the GROMACS manual for a description of the requirements that
must be satisfied by descriptions of simulation cells.


                 :-) GROMACS - gmx editconf, 2023-Homebrew (-:

Executable:   /opt/homebrew/bin/../Cellar/gromacs/2023/bin/gmx
Data prefix:  /opt/homebrew/bin/../Cellar/gromacs/2023
Working dir:  /Users/alexey/Dev/MD_conf
Command line:
  gmx editconf -f cal_104_fixed/CaCO3_substrate.pdb -o cal_104_fixed/CaCO3_substrate_init.gro


GROMACS reminds you: "Enthusiasm is the mother of effort, and without it nothing great was ever achieved." (Ralph Waldo Emerson)



0

In [512]:
# struct = read_gro(f'{folder}/CaCO3_substrate_init.gro')
struct = read_gro(f'{folder}/CaCO3_cell.gro')

In [513]:
for atom in struct.atoms:
    atom.mol_id = 1
    if atom.name[0:2] == 'Ca':
        atom.name = 'Ca'
        atom.mol_name = 'Caa'
    elif atom.name[0] == 'C':
        atom.name = 'CCa'
        atom.mol_name = 'CCaa'
    elif atom.name[0] == 'O':
        atom.name = 'OCa'
        atom.mol_name = 'OCaa'


In [514]:
struct.box = np.array([struct.box[0], struct.box[1], np.max([a.xyz[2] for a in struct.atoms])])

In [515]:
with open(f'{folder}/CaCO3_cell.gro', 'w') as f:
    f.write(write_gro(struct))

In [501]:
struct = read_gro(f'{folder}/CaCO3_cell.gro')

In [471]:
new_struct = Structure(box=struct.box)
id = 1
for atom in struct.atoms:
   if (atom.xyz[0] == 10.0 and atom.name == 'CCa'):
      continue
   new_atom = atom.copy()
   new_atom.id = id
   id += 1
   new_struct.atoms.append(new_atom)
     

In [504]:
struct.box = np.array([10.002368, 9.637720, np.max([a.xyz[2] for a in struct.atoms])])

struct.box

array([10.002368,  9.63772 ,  1.079   ])

In [492]:
with open(f'{folder}/CaCO3_substrate.gro', 'w') as f:
    f.write(write_gro(struct))

In [415]:
H = np.array([
    [1, 0, 0],
    [-1 / np.tan(np.deg2rad(angle[2])), 1 / np.sin(np.deg2rad(angle[2])), 0],
    [0, 0, 1]
])

struct = read_gro(f'{folder}/CaCO3_substrate_init.gro')
new_struct = Structure(box=np.array([struct.box[0], struct.box[2], struct.box[1]]))
id = 1

max = 0

for atom in struct.atoms:
    if np.any(np.abs(atom.xyz @ H - l / 2 - 0.01) >= l / 2):
        continue
    new_atom = atom.copy()
    new_atom.id = id
    new_atom.xyz = np.array([new_atom.xyz[0], new_atom.xyz[2], new_atom.xyz[1]])
    id += 1
    new_struct.atoms.append(new_atom)

In [416]:
with open(f'{folder}/CaCO3_substrate_pbc.gro', 'w') as f:
    f.write(write_gro(new_struct.apply_pbc()))

In [566]:
struct = read_gro(f'{folder}/CaCO3_substrate.gro')

In [567]:
folder

'cal_104_fixed'

In [568]:
names = defaultdict(int)
for atom in struct.atoms:
    names[atom.mol_name] += 1

with open(f'{folder}/calmol.itp', 'w') as f:
    for key, val in names.items():
        f.write(f'{key}\t{val}\n')

os.system(f'cp {folder}/CaCO3_substrate.gro {folder}/cal.gro')

0

In [569]:
names.keys()

dict_keys(['CCaa', 'OCaa', 'Caa'])

In [570]:
new_struct = Structure(box=struct.box)
id = 1
for key in names.keys():
    for a in struct.atoms:
        if a.mol_name == key:
            new_atom = a.copy()
            new_atom.id = id
            id += 1
            new_struct.atoms.append(new_atom)
        

with open(f'{folder}/cal.gro', 'w') as f:
    f.write(write_gro(new_struct))

---

In [483]:
(10.00237 + 9.63772) / 2 / 2.5

3.9280180000000002

In [394]:
3* 0.500680128 * np.sin(np.deg2rad(angle[2]))

1.3008051300541332

In [318]:
10 / 1.699012790

5.885770877569438

In [233]:
struct.box @ H

array([ 5.677585  , 10.01359771,  3.39803   ])

In [275]:
H = np.array([
    [1, 0, 0],
    [-1 / np.tan(np.deg2rad(angle[2])), 1 / np.sin(np.deg2rad(angle[2])), 0],
    [0, 0, 1]
])

np.matmul(struct.box, H)

array([15.02039885, 10.01359771,  3.39803   ])

In [88]:
struct.box = np.array([10.013602, 10.013602 * np.sin(np.deg2rad(120)), 3.398026])

In [90]:
struct = struct.apply_pbc()

In [302]:
angle = np.array(list(map(float, [cf['caco3']['_cell_angle_alpha'], cf['caco3']['_cell_angle_beta'], cf['caco3']['_cell_angle_gamma']])))
length = np.array(list(map(float, [cf['caco3']['_cell_length_a'], cf['caco3']['_cell_length_b'], cf['caco3']['_cell_length_c']]))) * 0.1

bais_x = np.array([1, 0, 0]) * length[0]
bais_y = np.array([np.cos(np.deg2rad(angle[2])), np.sin(np.deg2rad(angle[2])), 0]) * length[1]
bais_z = np.array([0, 0, 1]) * length[2]

In [337]:
struct = read_gro('xz/CaCO3_substrate_pbc.gro')

In [339]:
for atom in struct.atoms:
    atom.xyz = np.array([atom.xyz[0], atom.xyz[2], atom.xyz[1]])

struct.box = np.array([struct.box[0], struct.box[2], struct.box[1]])

In [340]:
with open('xz/CaCO3_substrate_pbc.gro', 'w') as f:
    f.write(write_gro(struct))

---

### Генерируем .itp для запуска

In [334]:
struct = read_gro('xz/CaCO3_substrate_pbc.gro')

names = defaultdict(int)
for atom in struct.atoms:
    names[atom.mol_name] += 1

with open('xz/calmol.itp', 'w') as f:
    for key, val in names.items():
        f.write(f'{key}\t{val}\n')

In [336]:
(10.01360 + 10.19408) / 2 / 2.5

4.041536

---

### Выравниваем заряд

In [534]:
CCaa =	3840
OCaa =	11520
Caa	 =  3840

Caa_charge = 1.6689
CCaa_charge = 0.9989
OCaa_charge = -0.889

print(np.float32(Caa_charge*Caa + CCaa_charge*CCaa + OCaa_charge*OCaa))

charge = np.float32(-(Caa_charge*Caa + CCaa_charge*CCaa) / OCaa)
print(charge)

print(np.float32(Caa_charge*Caa + CCaa_charge*CCaa + charge*OCaa))

3.072
-0.88926667
-3.5522462e-05


---

In [523]:
struct = read_gro('cal_104_fixed/CaCO3_cell.gro')

for a in struct.atoms:
    a.xyz[2] -= 0.09

with open('cal_104_fixed/CaCO3_cell.gro', 'w') as f:
    f.write(write_gro(struct))